<a href="https://colab.research.google.com/github/Roby290597/nlp_exercise/blob/main/bundestags_reden_analyse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import os 
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import requests

import sys
sys.path.append("extr")

from redner_extraction import extract_speeches, extract_all_speakers



In [23]:
from sentence_transformers import SentenceTransformer
import numpy as np  

In [ ]:
url = "https://www.bundestag.de/resource/blob/1115000/21032.xml" ### Bundestag Reden XML Datei vom November 2025 (Stand 30. November 2025 war es das aktuellste)
response = requests.get(url)

# XML in einen Tree parsen
tree = ET.ElementTree(ET.fromstring(response.content))

# Wurzel-Element abrufen
root = tree.getroot()



['vertrieb', 'herstellung', 'sitzung-ort', 'herausgeber', 'issn', 'wahlperiode', 'sitzung-nr', 'sitzung-datum', 'sitzung-start-uhrzeit', 'sitzung-ende-uhrzeit', 'sitzung-naechste-datum', 'start-seitennr']


In [29]:
import sys  
sys.path.append('C:/Users/brand/OneDrive/Desktop/AI_Selbststudium/ML_little/bundestag_reden/extr')  # Pfad zu deinem Modul

from redner_extraction import extract_speeches

reden = {}
redner = []
# Beispiel-Ausgabe
for speech in extract_speeches(root):
    #print(f"Redner: {speech['name']}\nRede: {speech['text'][:200]}...\n")
    if speech['name'] not in redner:
        redner.append(speech['name'])
        reden[speech['name']] = speech['text']
    else:
        print("DOPPELT:", speech['name'])
        reden[speech['name']] += "\n Nächste Rede:" + speech['text']

DOPPELT: Mandy Eißing
DOPPELT: Peter Bohnhof
DOPPELT: Mahmut Özdemir
DOPPELT: Klaus Wiener
DOPPELT: Mirze Edis


In [30]:
from redner_extraction import extract_all_speakers

redner_party = []
all_speakers = extract_all_speakers(root)
for speaker in all_speakers:
    print(speaker)
    redner_party.append((speaker['vorname'] + " " + speaker['nachname'], speaker['fraktion']))

{'vorname': 'Bärbel', 'nachname': 'Bas', 'fraktion': ''}
{'vorname': 'Hans-Jürgen', 'nachname': 'Goßner', 'fraktion': 'AfD'}
{'vorname': 'Wilfried', 'nachname': 'Oellers', 'fraktion': 'CDU/CSU'}
{'vorname': 'Ricarda', 'nachname': 'Lang', 'fraktion': 'BÜNDNIS\xa090/DIE GRÜNEN'}
{'vorname': 'Pascal', 'nachname': 'Meiser', 'fraktion': 'Die Linke'}
{'vorname': 'Dagmar', 'nachname': 'Schmidt', 'fraktion': 'SPD'}
{'vorname': 'Peter', 'nachname': 'Bohnhof', 'fraktion': 'AfD'}
{'vorname': 'Sandra', 'nachname': 'Carstensen', 'fraktion': 'CDU/CSU'}
{'vorname': 'Armin', 'nachname': 'Grau', 'fraktion': 'BÜNDNIS\xa090/DIE GRÜNEN'}
{'vorname': 'Nora', 'nachname': 'Seitz', 'fraktion': 'CDU/CSU'}
{'vorname': 'Leif-Erik', 'nachname': 'Holm', 'fraktion': 'AfD'}
{'vorname': 'Peter', 'nachname': 'Aumer', 'fraktion': 'CDU/CSU'}
{'vorname': 'Jan', 'nachname': 'Dieren', 'fraktion': 'SPD'}
{'vorname': 'Elisabeth', 'nachname': 'Winkelmeier-Becker', 'fraktion': 'CDU/CSU'}
{'vorname': 'Tobias Matthias', 'nachnam

In [31]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer("all-MiniLM-L6-v2")  # klein & schnell; gute Qualität

def embed(texts):
    # texts: str or list[str]
    return model.encode(texts, convert_to_numpy=True, normalize_embeddings=False)

def cosine_sim(a, b):
    # numerisch stabil: L2-normalisieren dann dot
    a = a / np.linalg.norm(a)
    b = b / np.linalg.norm(b)
    return float(np.dot(a, b))

# Beispiel
t1 = "Das ist ein Beispieltext."
t2 = "Das ist ein Beispieltext."
v1, v2 = embed([t1, t2])


print(v1)
print("Cosine similarity:", cosine_sim(v1, v2))


[ 1.71168018e-02  1.13473438e-01  5.30361310e-02  9.31795966e-03
  2.82452349e-02  8.40695724e-02  1.98806182e-01  1.01731032e-01
  7.63388574e-02 -8.89012888e-02 -1.22315744e-02 -1.29341751e-01
  1.60326235e-04 -4.19373438e-02  1.51388347e-02 -2.21811719e-02
 -3.17822094e-03  2.65002418e-02  6.52839802e-03  1.22141894e-02
 -3.07881571e-02 -3.47510241e-02  1.09352861e-02  4.81517687e-02
 -2.02234276e-02  6.78661987e-02 -1.50506978e-03 -3.79831530e-02
  5.63570224e-02  3.46733741e-02  6.91706082e-03  3.16232815e-02
  6.58472627e-02 -7.10361870e-03  3.11902128e-02 -2.09654458e-02
 -2.13662311e-02  3.43014635e-02  7.04338716e-04  2.82550715e-02
 -8.48518535e-02  4.67976145e-02 -4.18813713e-02 -1.98556874e-02
  6.72679469e-02  1.97251905e-02  4.20759991e-02  7.90748149e-02
 -4.22229953e-02  6.84830472e-02 -6.39509186e-02 -5.96090332e-02
 -1.96901411e-02  6.94304407e-02  4.75302972e-02 -2.41617355e-02
  2.32806709e-02 -2.67695840e-02 -4.52366434e-02 -2.66529657e-02
 -1.69616297e-03 -3.75755

In [4]:
from transformers import pipeline

# using pipeline API for summarization task
summarization = pipeline("summarization")


original_text = """
Im März 2025 habe ich mein Masterstudium in Artificial Intelligence & Data Science an der Heinrich-Heine-Universität Düsseldorf erfolgreich abgeschlossen. Dabei konnte ich fundierte Kenntnisse in Machine Learning, Datenverarbeitung und Datenvisualisierung erwerben, die ich nun gezielt in der Praxis einsetzen möchte. 
Praxisbezogene Erfahrungen sammelte ich am Forschungszentrum Jülich sowie beim Deutschen Zentrum für Luft- und Raumfahrt (DLR) in Sankt Augustin. Im Rahmen meiner Masterarbeit entwickelte und validierte ich ML-basierte Graphmodelle zur Analyse hydraulischer Systeme. Diese Arbeit verband die Verarbeitung komplexer technischer Daten mit der Entwicklung praxistauglicher Algorithmen. 
Ich bewerbe mich als AI-Engineer bei Thalia, weil mich die Möglichkeit begeistert, mit Generative AI echte Innovationen im Buchhandel mitzugestalten. Mit meinem Background in Machine Learning und Erfahrung mit Python, ML-Frameworks und Datenanalyse kann ich helfen, KI-Modelle zu entwickeln und produktiv einzusetzen. Ich arbeite strukturiert und verantwortungsbewusst und liebe es, technische Konzepte praktisch umzusetzen. Ich freue mich darauf, mit meiner Expertise zum digitalen Wandel bei Thalia beizutragen. 
Ich gelte als engagiert, lernbereit und teamorientiert. In bisherigen Stationen wurde meine Fähigkeit geschätzt, mich schnell in neue Themen einzuarbeiten, Ideen proaktiv einzubringen und konstruktiv mit Fach- und Führungskräften zusammenzuarbeiten. Auch wäre Ich bereit, bei einer erfolgreichen Bewerbungsprozess meinen Wohnort zu wechseln.  
Ich stehe Ihnen kurzfristig zur Verfügung und freue mich, Sie in einem persönlichen Gespräch von meiner Motivation und meinen Fähigkeiten zu überzeugen. 
Mit freundlichen Grüßen,  
Brandon (Roby) Hank 
"""

summary_text = summarization(original_text)[0]['summary_text']
print("Summary:", summary_text)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
c:\Users\brand\anaconda3\envs\job\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\brand\.cache\huggingface\hub\models--sshleifer--distilbart-cnn-12-6. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more det

Summary:  Brandon Hank: "Im März 2025 habe ich mein Masterstudium in Artificial Intelligence & Data Science an der Heinrich-Heine-Universität Düsseldorf erfolgreich abgeschlossen" "Ich bewerbe mich als AI-Engineer bei Thalia, weil mich die Möglichkeit begeistert"


In [14]:

summary_text = summarization(reden["Leif-Erik Holm"][:1000])[0]['summary_text']
print("Summary:", summary_text)

Summary:  Dieses Gesetz ist einfach nur Unfug. Es schadet den kleinen und mittleren Unternehmen in unserem Land. Wir haben es beim Verbrenner-Aus gesehen; wir sehen es immer noch. Heute bekommen es quasi vor den Latz geknallt. Es bewirkt genau das Gegenteil von dem, was Sie gestern eigentlich bewirken wollten. Es ist der nächste Klotz am Bein unserer ohnehin gebeutelten


In [16]:
for name, text in reden.items():
    print(f"Redner: {name}\nZusammenfassung: {summarization(text[:1000])[0]['summary_text']}\n")

Redner: Bärbel Bas
Zusammenfassung:  500 Milliarden Euro – 500 million Euro – ise viel Geld in die Hand genommen, um unser Land voranzubringen . Wir investieren in Kitas, in Schulen, in Krankenhäuser, in Klimaschutz. Wir stärken die kritische Infrastruktur, auch in Zeiten neuartiger Bedrohungen .

Redner: Hans-Jürgen Goßner
Zusammenfassung:  Tariftreuegesetz ist wieder einmal ein Paradebeispiel deutscher Bürokratiepolitik . Wer solche Widersprüche produziert, gehört nicht in die Regierung, sondern auf die Comedybühne .

Redner: Wilfried Oellers
Zusammenfassung:  Der Gesetzentwurf tangiert Artikel 9 Absatz 3 GG insoweit, als dass negative Koalitionsfreiheit davon betroffen ist . Es sollen gerade nicht durch niedrigere Lohnzahlungen günstigere Angebote auf dem Rücken der Beschäftigten abgegeben werden .

Redner: Ricarda Lang
Zusammenfassung:  Die AfD hetzt gegegen die Rechte von Arbeitnehmern, nur um sich dann im eigenen Wahlkreis wieder als die Partei des kleinen Mannes aufzuspielen . S

KeyboardInterrupt: 

In [19]:
from rouge_score import rouge_scorer
# Initialisierung des Scorers
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
# Vergleich von zwei Texten
scores = scorer.score(
   'Der schnelle braune Fuchs springt über den faulen Hund.',
   'Der schnelle braune Hund springt auf den Baum.'
)
print(scores)

{'rouge1': Score(precision=0.75, recall=0.6666666666666666, fmeasure=0.7058823529411765), 'rougeL': Score(precision=0.625, recall=0.5555555555555556, fmeasure=0.5882352941176471)}
